In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, Add, Input
from keras.utils import to_categorical

In [8]:
X_train = np.load("features_train.npy")
X_test = np.load("features_test.npy")
y_train = np.load("labels_train.npy")
y_test = np.load("labels_test.npy")

DATADIR_TRAIN = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/train"
DATADIR_TEST = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/test"

In [9]:
X_train = X_train / 255.0
X_test = X_test / 255.0
IMG_SIZE = 224

In [10]:
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [22]:
def identity_block(X, filters):
    f1, f2, f3 = filters
    # save the value to add later
    x_skip = X
    
    # Layer 1`
    Conv2D(f1, kernel_size = (1, 1), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Layer 2
    X = Conv2D(f2, kernel_size = (3,3), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    #layer 3
    X = Conv2D(f3, kernel_size = (1,1), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)

    # Add skip value to main path
    X = Add()([X, x_skip])
    X = Activation('relu')(X)
    
    
    return X

In [35]:
def convolution_block(X, filters, s):
    f1, f2, f3 = filters
    # save the value to add later
    x_skip = X
    
    # Layer 1
    X = Conv2D(f1, (1,1), strides = (s, s))(X) #remove padding to scale down
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    # Layer 2
    X = Conv2D(f2, (3,3), strides = (1, 1), padding = 'same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    #layer 3
    X = Conv2D(f3, kernel_size = (1,1), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    
    # Processing skip with conv(1,1)
    x_skip = Conv2D(f3, (1,1), strides = (s,s))(x_skip)
    x_skip = BatchNormalization(axis=3)(x_skip)
    
    # Add skip to main path
    X = Add()([X, x_skip])     
    X = Activation('relu')(X)
    
    return X

In [36]:
def ResNet50(shape = (IMG_SIZE, IMG_SIZE, 3), classes = 3):
    X_input = Input(shape)
   
    # STAGE 1: convolution w/ kernal size 7x7,  64 filters, with stride 2, followed by max pool
    X = Conv2D(64, kernel_size = (7,7), strides = 2, padding = 'same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    # STAGE 2
    X = convolution_block(X, filters = [64, 64, 256], s=1)
    X = identity_block(X, [64, 64, 256])
    X = identity_block(X, [64, 64, 256])
    
    # STAGE 3
    X = convolution_block(X, filters = [128, 128, 512], s=2)
    X = identity_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])
    
    # STAGE 4
    X = convolution_block(X, filters = [256, 256, 1024], s=2)
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    
    # STAGE 5
    X = convolution_block(X, filters = [512, 512, 2048], s=2)
    X = identity_block(X, [512, 512, 2048])
    X = identity_block(X, [512, 512, 2048])
    
    X = AveragePooling2D(2,2, padding='same')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    return model

In [37]:
model = ResNet50(shape = (IMG_SIZE,IMG_SIZE, 3), classes = 3)

In [38]:
model.compile(
    optimizer='adam', # optimizer
    loss='categorical_crossentropy', # loss function to optimize 
    metrics=['accuracy'] # metrics to monitor
)

In [39]:
model.fit(
    X_train, y_train,
    epochs = 5,
    batch_size = 32,
    )

Epoch 1/5
8/8 [==============================] - 26s 3s/step - loss: 9.5963 - accuracy: 0.4422
Epoch 2/5
8/8 [==============================] - 23s 3s/step - loss: 2.7395 - accuracy: 0.6892
Epoch 3/5
8/8 [==============================] - 23s 3s/step - loss: 0.7925 - accuracy: 0.7211
Epoch 4/5
8/8 [==============================] - 23s 3s/step - loss: 0.5838 - accuracy: 0.7928
Epoch 5/5
8/8 [==============================] - 23s 3s/step - loss: 0.5092 - accuracy: 0.8526


In [40]:
preds = model.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3/3 [==============================] - 2s 453ms/step - loss: 38.9808 - accuracy: 0.3939
Loss = 38.98080062866211
Test Accuracy = 0.39393940567970276
